# Alzheimer Classification
## Stochastic Gradient Descent

In [1]:
import pandas as pd 
data = pd.read_csv("Alzheimer's.csv")
#Converted the group to dummy variables
data.loc[data.Group=='Nondemented', 'Group'] = 0
data.loc[data.Group=='Demented', 'Group'] = 1
data.loc[data.Group=='Converted', 'Group'] = 2

#Converted the M/F to dummy variables
data.loc[data.Sex=='M', 'Sex'] = 0
data.loc[data.Sex=='F', 'Sex'] = 1
import numpy as np 
from sklearn.model_selection import train_test_split

#Attempting PCA on data
#Hand is completely useless as it is identical for all samples
data_drop = data.drop(['Hand','Visit','Subject ID','MRI ID'], axis = 1) #axis = 1 means to drop column not row

#get rid of row 360 and 359 bc they are missing alot of data (both SES and MMS)
data_drop = data_drop.drop([360, 359])


#delete all data points that dont have SES in them (this is where they have NaN)
data_drop = data_drop.dropna()
group = data_drop[['Group']] 
# give it a label type
group = group.astype('int')

#dementia status is what we want to predict - change this to single target 
data_drop = data_drop.drop(['Group'], axis = 1) #axis = 1 means to drop column not row
#get a list of columns in pandas object 
names_of_data = data_drop.columns.tolist()

In [2]:
# split the data

#shuffle = false prevents data split being different everytime
X_train, X_test, y_train, y_test = train_test_split(data_drop, group, test_size=0.2, shuffle = False)

#split test into validate and test, again making sure the data is always the same for consistency
X_test, X_val, y_test, y_val = train_test_split(X_train, y_train, test_size=0.25, shuffle = False)

#Normalizing the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#running the actual PCA
from sklearn.decomposition import PCA

pca = PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

y_train = y_train.astype('int').to_numpy().ravel()
y_test = y_test.astype('int').to_numpy().ravel()

data_drop.loc[354]
data_drop.loc[355]

MR Delay                                 652
Sex                                        0
Age                                       81
EDUC                                      20
SES                                        1
Mini Mental State                         26
Clinical Dementia Rating                 0.5
Estimated total Intracranial Volume     1556
Normalize Whole Brain Volume           0.691
Atlas Scaling Factor                   1.128
Name: 355, dtype: object

In [3]:
from sklearn.linear_model import SGDClassifier
model = SGDClassifier(loss="hinge", penalty="l2", max_iter=100)
model.fit(X_train, y_train)
#SGDClassifier(max_iter=5)
y_pred = model.predict(X_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test.astype('int'), y_pred))

0.9198113207547169
